In [1]:
%load_ext autoreload
%autoreload 2
from confignb import *

INFO:2026-01-23 05:20:50,260:jax._src.xla_bridge:834: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
05:20:50: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [2]:
X, y, pts, ks = h.load_data(DATA_DIR / 'ASCADv_clean.h5', as_df=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import SelectFromModel
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

rfc = RFC(max_depth=5, min_samples_leaf=10, n_jobs=-1, random_state=SEED)

pl_nystroem = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    SelectFromModel(rfc, max_features=200),
    Nystroem(kernel='poly', degree=2, n_components=20, gamma=0.1, random_state=42, n_jobs=-1), # zamiast kpca
    LogisticRegression(C=0.5, max_iter=1000, random_state=SEED, n_jobs=-1)
)

score = np.mean(h.cv(SEED, pl_nystroem, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler

rfc = RFC(max_depth=5, min_samples_leaf=10, n_jobs=-1, random_state=SEED) # najmniejsza głębokość dająca dobre wyniki

pl_poly = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    SelectFromModel(rfc, max_features=200),
    # interaction_only=False nie pomaga, tak samo degree=(1, 2)
    PolynomialFeatures(degree=(2, 2), interaction_only=True, include_bias=False),
    SelectKBest(f_classif, k=20),
    LogisticRegression(C=0.5, max_iter=300, random_state=SEED), # zmniejszanie C pogarsza wyniki
    # LogisticRegression(l1_ratio=1, C=0.5, random_state=SEED,  # --- takie same wyniki, o wiele wolniejsza
    #                    solver='saga', verbose=1, tol=1e-3),
)

score = np.mean(h.cv(SEED, pl_poly, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

In [ ]:
from functools import partial

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import (
    RFE,
    SelectFromModel,
    SelectKBest,
    f_classif,
)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler

SimpleRFC = partial(RFC, max_depth=5, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


pl_turbo01 = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    SelectFromModel(SimpleRFC(), max_features=150),
    PolynomialFeatures(degree=(2, 2), interaction_only=True, include_bias=False),
    SelectKBest(f_classif, k=250),
    # SelectKBest(score_func=mi_scorer, k=50),  # --- ram umarł
    RFE(estimator=SimpleRFC(), n_features_to_select=100, step=10),
    LogisticRegression(C=0.5, max_iter=500, random_state=SEED),
    # LDA(solver='lsqr', shrinkage='auto',)     # --- gorsze wyniki
)

score = np.mean(h.cv(SEED, pl_turbo01, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

In [ ]:
from functools import partial

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler

SimpleRFC = partial(RFC, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


pl_turbo_simple = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    SelectFromModel(SimpleRFC(max_depth=5), max_features=200),

    PolynomialFeatures(degree=(2, 2), interaction_only=True, include_bias=False),
    SelectKBest(f_classif, k=500),

    SelectFromModel(SimpleRFC(max_depth=5), max_features=100),

    # RFE(estimator=SimpleRFC(), n_features_to_select=100, step=5),
    LogisticRegression(C=0.5, max_iter=500, random_state=SEED),
)

score = np.mean(h.cv(SEED, pl_turbo_simple, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

22:15:46: [1] Mean PI: 0.137
22:16:34: [2] Mean PI: 0.124
22:17:07: [3] Mean PI: 0.131
22:17:42: [4] Mean PI: 0.137
22:18:15: [5] Mean PI: 0.132
22:18:49: [6] Mean PI: 0.132
22:19:24: [7] Mean PI: 0.140
22:19:58: [8] Mean PI: 0.132
22:20:34: [9] Mean PI: 0.140
22:21:06: [10] Mean PI: 0.128
22:21:06: Mean PI: 1.33e-01


In [ ]:
from functools import partial

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    PolynomialFeatures,
    RobustScaler,
)

SimpleRFC = partial(RFC, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


pl_turbo_simple = make_pipeline(
    # StandardScaler(),     # --- działa beznadziejnie
    # RobustScaler(),
    RobustScaler(quantile_range=(5, 95)), # --- działa najlepiej
    SelectFromModel(SimpleRFC(max_depth=5), max_features=200), # 200 daje najlepsze wyniki

    PolynomialFeatures(degree=(2, 2), interaction_only=True, include_bias=False),
    SelectKBest(f_classif, k=400),

    SelectFromModel(SimpleRFC(max_depth=5), max_features=100),
    # RFE(SimpleRFC(max_depth=5), n_features_to_select=20, step=10), # --- w sumie niepotrzebne

    LogisticRegression(C=0.5, max_iter=500, random_state=SEED),
)

score = np.mean(h.cv(SEED, pl_turbo_simple, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

23:06:02: [1] Mean PI: 0.126


: 

In [ ]:
from functools import partial

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures, RobustScaler

SimpleRFC = partial(RFC, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


def random_poly_features(m, triplets=False, seed=SEED):
    def fn(X, y=None):
        k = X.shape[1]
        X2 = X @ h.sample_doublets_onehot(k, m, seed)
        if triplets:
            X3 = X @ h.sample_triplets_onehot(k, m, seed)
            return np.concat([X2, X3], axis=1)
        else:
            return X2
    return fn


pl_turbo_simple = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    SelectFromModel(SimpleRFC(max_depth=5), max_features=200),

    # FunctionTransformer(random_poly_features(m=10000, seed=SEED)),
    PolynomialFeatures(degree=(2, 2), interaction_only=True, include_bias=False),

    SelectKBest(f_classif, k=400),
    SelectFromModel(SimpleRFC(max_depth=5), max_features=100),

    LogisticRegression(C=0.5, max_iter=500, random_state=SEED),
)

score = np.mean(h.cv(SEED, pl_turbo_simple, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

01:28:35: [1] Mean PI: 0.139


KeyboardInterrupt: 

In [ ]:
from functools import partial

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

SimpleRFC = partial(RFC, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


pl_turbo_simple = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    # SelectFromModel(SimpleRFC(max_depth=5), max_features=200),

    h.RandomCombinations(m2=15000, seed=SEED),
    # PolynomialFeatures(degree=(2, 2), interaction_only=True, include_bias=False),

    SelectKBest(f_classif, k=500),
    SelectFromModel(SimpleRFC(max_depth=5), max_features=100),

    LogisticRegression(C=0.5, max_iter=500, random_state=SEED),
    # SimpleRFC(max_depth=5)
)

score = np.mean(h.cv(SEED, pl_turbo_simple, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

In [4]:
from functools import partial

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler

SimpleRFC = partial(RFC, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


pl_turbo_simple = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    h.RandomCombinations(m2=15000, topk=200, seed=SEED, weight_model=SimpleRFC(max_depth=5)),

    SelectKBest(f_classif, k=500),
    SelectFromModel(SimpleRFC(max_depth=5), max_features=100),
    LogisticRegression(C=0.5, max_iter=500, random_state=SEED)
)

score = np.mean(h.cv(SEED, pl_turbo_simple, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

04:45:42: [1] Mean PI: 0.142
04:46:29: [2] Mean PI: 0.130
04:47:14: [3] Mean PI: 0.132
04:47:57: [4] Mean PI: 0.136
04:48:39: [5] Mean PI: 0.127
04:49:12: [6] Mean PI: 0.127
04:49:49: [7] Mean PI: 0.136
04:50:28: [8] Mean PI: 0.124
04:51:30: [9] Mean PI: 0.132
04:52:52: [10] Mean PI: 0.131
04:52:52: Mean PI: 1.32e-01


In [ ]:
from functools import partial

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler

SimpleRFC = partial(RFC, min_samples_leaf=10, n_jobs=-1, random_state=SEED)


pl_turbo_simple = make_pipeline(
    RobustScaler(quantile_range=(5, 95)),
    SelectFromModel(SimpleRFC(max_depth=5), max_features=300),
    CatBoostClassifier(
        task_type="GPU", devices='0', random_seed=SEED,
        depth=5, iterations=500, learning_rate=0.1,
        border_count=32,
    )
)

score = np.mean(h.cv(SEED, pl_turbo_simple, X, y, pts, ks))
logging.info(f"Mean PI: {score:.2e}")

0:	learn: 5.5450567	total: 318ms	remaining: 2m 38s
1:	learn: 5.5426517	total: 634ms	remaining: 2m 37s
2:	learn: 5.5411467	total: 946ms	remaining: 2m 36s
3:	learn: 5.5410823	total: 1.25s	remaining: 2m 35s
4:	learn: 5.5394504	total: 1.56s	remaining: 2m 34s
5:	learn: 5.5375698	total: 1.86s	remaining: 2m 33s
6:	learn: 5.5360085	total: 2.17s	remaining: 2m 33s
7:	learn: 5.5356979	total: 2.48s	remaining: 2m 32s
8:	learn: 5.5351797	total: 2.82s	remaining: 2m 33s
9:	learn: 5.5340985	total: 3.15s	remaining: 2m 34s
10:	learn: 5.5333253	total: 3.46s	remaining: 2m 33s
11:	learn: 5.5330153	total: 3.77s	remaining: 2m 33s
12:	learn: 5.5330028	total: 4.07s	remaining: 2m 32s
13:	learn: 5.5329866	total: 4.38s	remaining: 2m 31s
14:	learn: 5.5329797	total: 4.68s	remaining: 2m 31s
15:	learn: 5.5324566	total: 4.99s	remaining: 2m 30s
16:	learn: 5.5321203	total: 5.3s	remaining: 2m 30s
17:	learn: 5.5314234	total: 5.62s	remaining: 2m 30s
18:	learn: 5.5311747	total: 5.93s	remaining: 2m 30s
19:	learn: 5.5308028	to

05:23:37: [1] Mean PI: 0.006


0:	learn: 5.5443087	total: 316ms	remaining: 2m 37s
1:	learn: 5.5437575	total: 624ms	remaining: 2m 35s
2:	learn: 5.5429087	total: 953ms	remaining: 2m 37s
3:	learn: 5.5417131	total: 1.27s	remaining: 2m 37s
4:	learn: 5.5406981	total: 1.57s	remaining: 2m 35s
5:	learn: 5.5406519	total: 1.87s	remaining: 2m 34s
6:	learn: 5.5403762	total: 2.17s	remaining: 2m 32s
7:	learn: 5.5401050	total: 2.47s	remaining: 2m 32s
8:	learn: 5.5393625	total: 2.78s	remaining: 2m 31s
9:	learn: 5.5364669	total: 3.1s	remaining: 2m 32s
10:	learn: 5.5361887	total: 3.4s	remaining: 2m 31s
11:	learn: 5.5354769	total: 3.71s	remaining: 2m 30s
12:	learn: 5.5342650	total: 4.04s	remaining: 2m 31s
13:	learn: 5.5336300	total: 4.35s	remaining: 2m 31s
14:	learn: 5.5331950	total: 4.66s	remaining: 2m 30s
15:	learn: 5.5329444	total: 4.96s	remaining: 2m 30s
16:	learn: 5.5306319	total: 5.29s	remaining: 2m 30s
17:	learn: 5.5298925	total: 5.6s	remaining: 2m 30s
18:	learn: 5.5289712	total: 5.92s	remaining: 2m 29s
19:	learn: 5.5277550	tota

KeyboardInterrupt: 

Exception ignored in: '_catboost._WriteLog'
Traceback (most recent call last):
  File "/home/notuser/workspace/SCA/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3051, in write
    def write(data, *args, **kwargs):
KeyboardInterrupt: 


56:	learn: 5.5089812	total: 17.8s	remaining: 2m 18s
57:	learn: 5.5087894	total: 18.1s	remaining: 2m 18s
58:	learn: 5.5085694	total: 18.4s	remaining: 2m 17s
59:	learn: 5.5078812	total: 18.7s	remaining: 2m 17s
60:	learn: 5.5078781	total: 19s	remaining: 2m 17s
61:	learn: 5.5073750	total: 19.4s	remaining: 2m 16s
62:	learn: 5.5067719	total: 19.7s	remaining: 2m 16s
63:	learn: 5.5062369	total: 20s	remaining: 2m 16s
64:	learn: 5.5056788	total: 20.3s	remaining: 2m 15s
65:	learn: 5.5056762	total: 20.6s	remaining: 2m 15s
66:	learn: 5.5053612	total: 20.9s	remaining: 2m 15s
67:	learn: 5.5053556	total: 21.2s	remaining: 2m 14s
68:	learn: 5.5051194	total: 21.5s	remaining: 2m 14s
69:	learn: 5.5047131	total: 21.8s	remaining: 2m 14s
70:	learn: 5.5043050	total: 22.1s	remaining: 2m 13s
71:	learn: 5.5030962	total: 22.5s	remaining: 2m 13s
72:	learn: 5.5027231	total: 22.8s	remaining: 2m 13s
73:	learn: 5.5025144	total: 23.1s	remaining: 2m 13s
74:	learn: 5.5025150	total: 23.4s	remaining: 2m 12s
75:	learn: 5.502